## Imports and Setup

In [ ]:
import os
import sys

import arviz as av
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from plots import *
from threeML import update_logging_level

update_logging_level("FATAL")

parent_dir = os.path.abspath("..")
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from zusammen import DataSet

sns.set_theme(context="paper")

mpl.use("pgf")
pgf_with_latex = {
    "text.usetex": True,
    "font.family": "serif",
    "axes.labelsize": 10,
    "font.size": 10,
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "pgf.texsystem": "lualatex",
    "pgf.rcfonts": "False",
    "pgf.preamble": "\n".join([r"\usepackage{siunitx}", r"\DeclareSIUnit{\erg}{erg}"]),
}
mpl.rcParams.update(pgf_with_latex)

width = 455. / 72.27
height = width / 1.61803398875

In [ ]:
# data_folder = "simulation/"
# data_name = "data_2_sig_5"
# survey_name = "survey_2"

# inference_folder = "inference/"
# inference_name = "simulated_2_sig_5_1000"
# inference_name = "simulated_relaxed_2_sig_5_1000"
# inference_name = "simulated_global_2_sig_5_1000"


data_folder = "real_data/"
data_name = "data"

inference_folder = "inference/"
inference_name = "real_sig_5_1000"
# inference_name = "real_relaxed_3_sig_5_1000"
# inference_name = "real_global_sig_5_1000"

In [ ]:
plots_folder = "/Users/chrobin/LRZ Sync+Share/Uni/Bachelorarbeit/Thesis/figures/"
if "real" in data_folder:
    model = "real"
else:
    model = "simulated"
if "relaxed" in inference_name:
    model += "_relaxed"
elif "global" in inference_name:
    model += "_global"

ds = DataSet.from_hdf5_file(data_folder + data_name + ".h5")
data = ds.to_stan_dict()
res = av.from_netcdf(inference_folder + inference_name + ".nc")

assert data["N_intervals"] == res.posterior.alpha.shape[2]

## Basics

### Band function

In [ ]:
%matplotlib widget
fig = plot_band(width)
# fig.savefig(plots_folder + "band.pdf")

### Light Curve and Spectrum

In [ ]:
# %matplotlib widget
fig = plot_light_curve_basics(model=model, data_folder=data_folder, grb_name="GRB160509374", detector="n1", width=width, height=height, siunitx=True)
fig.savefig(plots_folder + "lightcurve_ex.pdf")

In [ ]:
fig = plot_spectrum_basics(model=model, data_folder=data_folder, grb_name = "GRB160509374", detector="n1", width=width, height=height, siunitx=True)
fig.savefig(plots_folder + "spectrum_ex.pdf")

## Methods

### Weak Light Curve

In [ ]:
# %matplotlib widget
fig = plot_weak_light_curve(model=model, data_folder=data_folder, survey_name=survey_name, width=width, siunitx=True)
fig.savefig(plots_folder + "lc_insignificant.pdf")

## Results

### Light Curves

In [ ]:
%matplotlib widget

kwargs = {}
if "simulated" in model:
    kwargs["survey_name"] = survey_name
    kwargs["data_name"] = data_name
else:
    kwargs["grb_name"] = "GRB160509374"
fig = plot_light_curve(model=model, data_folder=data_folder, width=width, **kwargs)
# fig.savefig(plots_folder + "lc_" + ["real" if "real" in model else "simulated"][0] + ".pdf")

### PPC

In [ ]:
%matplotlib widget
fig = plot_ppc(model, ds, data, res, width)
# plt.savefig(plots_folder + "ppc_" + model + ".pdf")

### Corner Plot

In [ ]:
%matplotlib widget

fig = plot_corner(model, res, width)
fig.savefig(plots_folder + "corner_" + model + ".pdf")

### Violin Plot

In [ ]:
%matplotlib widget
fig_gamma, fig_Nrest = plot_violin(model, res, width, height)
# fig.savefig(plots_folder + "violin_gamma_" + model + ".pdf")

### Trace Plot

In [ ]:
%matplotlib widget
fig = plot_trace(model, res, width)
# fig.savefig(plots_folder + "trace_" + model + ".pdf")

### GC

In [ ]:
%matplotlib widget
fig = plot_gc(model, data, res, width, height)
# fig.savefig(plots_folder + "gc_" + model + ".pdf")

### Correlation of Hyperparameters

In [ ]:
%matplotlib widget
fig = plot_gc_kde(model, res, width)
# fig.savefig(plots_folder + "meta_" + model + ".pdf")